# Transformer Model for Electricity Demand Forecasting

This script implements a PyTorch Transformer model for time series forecasting of electricity demand.
The model uses attention mechanisms to capture temporal dependencies in the data.

## HOW TO RUN THIS FILE:

1. **Prerequisites**: Ensure the processed dataset `complete_data.csv` is available in `../../data/processed/`
2. **Run All Cells**: Execute cells sequentially from top to bottom using "Run All" or run each cell individually
3. **Dependencies**: All required packages will be automatically installed in the initial section of the python script
4. **Outputs**: The script generates performance metrics, feature importance analysis, and visualisation plots
5. **Results**: Final model performance and predictions are saved in variables accessible throughout the session

In [1]:
%pip install torch

  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/888.0 MB ? eta -:--:--Requirement already satisfied: MarkupSafe>=2.0 in /home/nwadmin/ZZSC9020-Group_F/.venv/lib/python3.10/site-packages (from jinja2->torch) (3.0.2)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.0/888.0 MB 12.4 MB/s  0:00:39m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.0/888.0 MB 12.4 MB/s  0:00:39
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 14.6 MB/s  0:00:23m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 14.6 MB/s  0:00:23
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 17.1 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 17.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#imports
import torch
import torch.nn as nn
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
import os
import shap
from sklearn.metrics import r2_score
from matplotlib.gridspec import GridSpec

#global variables
patience = 20
num_epochs = 100
one_hot_features = ['is_summer', 'is_autumn', 'is_winter', 'is_spring',
                    'is_monday', 'is_tuesday', 'is_wednesday', 'is_thursday', 'is_friday', 'is_saturday', 'is_sunday',
                    'is_weekday', 'is_weekend',
                    'is_jan', 'is_feb', 'is_mar', 'is_apr', 'is_may', 'is_jun', 'is_jul', 'is_aug', 'is_sep', 'is_oct', 'is_nov', 'is_dec'
                    ]

#load and prepare data
def prepare_data(params):
    # 1. Load data
    # Get the repository root directory (two levels up from this file)
    current_dir = os.path.dirname(os.path.abspath(__file__))
    repo_root = os.path.dirname(os.path.dirname(current_dir))
    DATA_DIR = os.path.join(repo_root, 'data')

    if params["dataset"] == "2016-2019":
        data = pd.read_csv(os.path.join(DATA_DIR, 'processed/processed2.csv'))
        datetimes = pd.to_datetime(data['datetime_au'])
    elif params["dataset"] == "2010-2019":
        data = pd.read_csv(os.path.join(DATA_DIR, 'processed/processed_full.csv'))
        datetimes = pd.to_datetime(data['datetime_au'], dayfirst=True)

    # Select desired features
    data = data[params['features']]

    # Separate one-hot and continuous features
    continuous_features = [f for f in params['features'] if f not in one_hot_features]
    one_hot_feats = [f for f in params['features'] if f in one_hot_features]

    # Scale only continuous features
    scaler = StandardScaler()
    scaled_continuous = scaler.fit_transform(data[continuous_features].values)
    scaled_continuous = torch.FloatTensor(scaled_continuous)

    # Get one-hot features as tensor
    one_hot_tensor = torch.FloatTensor(data[one_hot_feats].values)

    # Concatenate scaled continuous and one-hot features
    scaled_data = torch.cat([scaled_continuous, one_hot_tensor], dim=1)

    # 3. Create sequences and targets
    sequences = []
    targets = []

    for i in range(len(scaled_data) - params["seq_length"]):
        sequences.append(scaled_data[i:i+params["seq_length"]])  # sequence of 7 days
        targets.append(scaled_data[i+params["seq_length"], 0])   # predict next day's demand (column 0)

    sequences = torch.stack(sequences)
    targets = torch.FloatTensor(targets).unsqueeze(1)  # shape: (n, 1)

    return sequences, targets, datetimes, scaler

def create_sequences(data, seq_length):
    """Create sequences of length seq_length from the data"""
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        sequences.append(seq)
    return torch.stack(sequences)

def create_targets(data, seq_length):
    """Create targets for each sequence (the next demand value after the sequence)"""
    targets = []
    for i in range(seq_length, len(data)):
        target = data[i, 0]  # Assuming demand is the first feature
        targets.append(target)
    return torch.stack(targets)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0) # Shape: (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return x

class TransformerModel(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_layers, dim_feedforward, dropout=0.1, activation='relu'):
        super(TransformerModel, self).__init__()
        self.d_model = d_model
        self.input_projection = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation=activation,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_layer = nn.Linear(d_model, 1)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        src_projected = self.input_projection(src) * math.sqrt(self.d_model)
        src_projected = self.pos_encoder(src_projected)
        encoder_output = self.transformer_encoder(src_projected, mask=src_mask, src_key_padding_mask=src_key_padding_mask)
        last_time_step_output = encoder_output[:, -1, :]
        output = self.output_layer(last_time_step_output)
        return output

def train_transformer_model(sequences, targets, input_dim, datetimes, params):
    """Train the Transformer model"""
    datetimes = pd.to_datetime(datetimes)

    if params["train_test_split"] == "80:20":
        X_train, X_val, y_train, y_val = train_test_split(
        sequences, targets, test_size=0.2, shuffle=False
        )
    elif params["train_test_split"] == "prior:2019":
        train_mask = datetimes.dt.year != 2019
        test_mask  = datetimes.dt.year == 2019
        seq_train_mask = train_mask[params['seq_length']:].to_numpy()
        seq_test_mask  = test_mask[params['seq_length']:].to_numpy()
        X_train, y_train = sequences[seq_train_mask], targets[seq_train_mask]
        X_val,   y_val   = sequences[seq_test_mask], targets[seq_test_mask]
    
    X_train_tensor = torch.FloatTensor(X_train)
    X_val_tensor   = torch.FloatTensor(X_val)
    y_train_tensor = torch.FloatTensor(y_train)
    y_val_tensor   = torch.FloatTensor(y_val)
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset   = TensorDataset(X_val_tensor, y_val_tensor)
    train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=params['batch_size'], shuffle=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = TransformerModel(
        input_dim=input_dim,
        d_model=params["transformer_encoder_layer_params"]['d_model'],
        nhead=params["transformer_encoder_layer_params"]['nhead'],
        num_layers=params["transformer_layer_params"]['num_layers'],
        dim_feedforward=params["transformer_encoder_layer_params"]['dim_feedforward'],
        dropout=params["transformer_encoder_layer_params"]['dropout'],
        activation=params["transformer_encoder_layer_params"]['activation']
    ).to(device)

    # These are pretty standard choices
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params["learning_rate"], weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)

    best_val_loss = float('inf')
    epochs_no_improve = 0
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * batch_X.size(0)
        
        train_loss = train_loss / len(train_loader.dataset)
        train_losses.append(train_loss)
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                predictions = model(batch_X)
                loss = criterion(predictions, batch_y)
                val_loss += loss.item() * batch_X.size(0)
        
        val_loss = val_loss / len(val_loader.dataset)
        val_losses.append(val_loss)
        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        
        if epochs_no_improve >= patience:
            break

    # Return train/test splits for postprocess
    return model, train_losses, val_losses, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor

def evaluate_model(model, X_val, y_val, device):
    model.eval()
    with torch.no_grad():
        X_val_tensor = torch.FloatTensor(X_val).to(device)
        predictions = model(X_val_tensor)
        predictions = predictions.cpu().numpy()
    return predictions


def postprocess(model, X_train, y_train, X_test, y_test, scaler, train_losses, val_losses, params, visualise=False):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Get continuous feature count
    continuous_features = [f for f in range(len(params['features'])) if f not in [params['features'].index(f) for f in one_hot_features if f in params['features']]]
    n_cont = len(continuous_features)

    # Train set
    train_predictions = evaluate_model(model, X_train.numpy(), y_train.numpy(), device)
    dummy_train = np.zeros((len(train_predictions), n_cont))
    dummy_train[:, 0] = train_predictions.flatten()
    train_predictions_original = scaler.inverse_transform(dummy_train)[:, 0]
    dummy_train[:, 0] = y_train.numpy().flatten()
    train_targets_original = scaler.inverse_transform(dummy_train)[:, 0]
    train_mse = np.mean((train_predictions_original - train_targets_original) ** 2)
    train_mae = np.mean(np.abs(train_predictions_original - train_targets_original))
    train_rmse = np.sqrt(train_mse)
    train_mape = np.mean(np.abs((train_predictions_original - train_targets_original) / train_targets_original)) * 100

    # Test set
    test_predictions = evaluate_model(model, X_test.numpy(), y_test.numpy(), device)
    dummy_test = np.zeros((len(test_predictions), n_cont))
    dummy_test[:, 0] = test_predictions.flatten()
    test_predictions_original = scaler.inverse_transform(dummy_test)[:, 0]
    dummy_test[:, 0] = y_test.numpy().flatten()
    test_targets_original = scaler.inverse_transform(dummy_test)[:, 0]
    test_mse = np.mean((test_predictions_original - test_targets_original) ** 2)
    test_mae = np.mean(np.abs(test_predictions_original - test_targets_original))
    test_rmse = np.sqrt(test_mse)
    test_mape = np.mean(np.abs((test_predictions_original - test_targets_original) / test_targets_original)) * 100


    results = {
        'train_mse': train_mse,
        'train_mae': train_mae,
        'train_rmse': train_rmse,
        'train_mape': train_mape,
        'test_mse': test_mse,
        'test_mae': test_mae,
        'test_rmse': test_rmse,
        'test_mape': test_mape,
        'train_losses': train_losses,
        'val_losses': val_losses,
        'test_targets_original': test_targets_original,
        'test_predictions_original': test_predictions_original
    }
    return results

def train_model(params):
    input_dim = len(params['features'])
    sequences, targets, datetimes, scaler_X = prepare_data(params)
    # Get train/test splits from train_transformer_model
    model, train_losses, val_losses, X_train, y_train, X_test, y_test = train_transformer_model(
        sequences, targets, input_dim, datetimes, params
    )
    results = postprocess(model, X_train, y_train, X_test, y_test, scaler_X, train_losses, val_losses, params, params['visualise'])
    return results, model, X_train, y_train, X_test, y_test

def median_model(params, runs):
    # Calculate median MAPE over 'runs' runs, saving all model parameters for each run
    models_list = []
    # Run 'runs' times, save all results
    for i in range(runs):
        results, model, X_train, y_train, X_test, y_test = train_model(params)
        models_list.append({
            'results': results,
            'model': model,
            'X_train': X_train,
            'y_train': y_train,
            'X_test': X_test,
            'y_test': y_test,
        })
    # Find the median MAPE index
    # Find the median MAPE index robustly (works for even or odd runs)
    test_mapes = [entry['results']['test_mape'] for entry in models_list]
    median_mape = np.median(test_mapes)
    # Find the index of the model whose test_mape is closest to the median
    median_idx = np.argmin(np.abs(np.array(test_mapes) - median_mape))
    median_entry = models_list[median_idx]
    # Visualise only the best (median) model
    if params['visualise']:
        visualise_model_results(median_entry['results'], runs)
    return median_entry

def visualise_model_results(results, runs):
    print(f"Median Model Results after {runs} runs:")
    print(f"\nTrain Set Metrics:")
    print(f"MSE: {results['train_mse']:.4f}")
    print(f"MAE: {results['train_mae']:.4f}")
    print(f"RMSE: {results['train_rmse']:.4f}")
    print(f"MAPE: {results['train_mape']:.4f}")

    print(f"\nTest Set Metrics:")
    print(f"MSE: {results['test_mse']:.4f}")
    print(f"MAE: {results['test_mae']:.4f}")
    print(f"RMSE: {results['test_rmse']:.4f}")
    print(f"MAPE: {results['test_mape']:.4f}")

    # Plot results (test set)
    plot_results(results['train_losses'], results['val_losses'], results['test_targets_original'], results['test_predictions_original'])

def plot_results(train_losses, val_losses, y_val, predictions):
    fig = plt.figure(figsize=(22, 7))
    gs = GridSpec(1, 3, width_ratios=[1, 1, 2])  # 25%, 25%, 50%

    # 1. Predicted vs Actual (Scatter)
    ax0 = fig.add_subplot(gs[0])
    ax0.scatter(y_val, predictions, color='blue', alpha=0.7, s=20)
    ax0.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], 'r--', label='Perfect Prediction')
    r2 = r2_score(y_val, predictions)
    ax0.text(0.05, 0.95, f'$R^2$ = {r2:.4f}', transform=ax0.transAxes, fontsize=13, verticalalignment='top', bbox=dict(boxstyle="round", fc="w"))
    ax0.set_xlabel('Actual Values')
    ax0.set_ylabel('Predicted Values')
    ax0.set_title('Predicted vs Actual (Test Data)')
    ax0.legend()
    ax0.grid(True)

    # 2. Residuals Plot
    ax1 = fig.add_subplot(gs[1])
    residuals = predictions - y_val
    ax1.scatter(predictions, residuals, color='green', alpha=0.7, s=20)
    ax1.axhline(0, color='red', linestyle='--')
    ax1.set_xlabel('Predicted Values')
    ax1.set_ylabel('Residuals')
    ax1.set_title('Residuals Plot')
    ax1.grid(True)

    # 3. Time Series Comparison (first 365 points)
    ax2 = fig.add_subplot(gs[2])
    ax2.plot(y_val[-365:], label='Actual', color='blue')
    ax2.plot(predictions[-365:], label='Predicted', color='red')
    ax2.set_xlabel('Time Index')
    ax2.set_ylabel('Demand Values')
    ax2.set_title('Time Series Comparison (365 points)')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.savefig('training_results.png', dpi=300, bbox_inches='tight')
    plt.show()

def explain_transformer_feature_importance(model, X_train, X_test, params):
    # Flatten sequences for SHAP
    X_train_flat = X_train.numpy().reshape(X_train.shape[0], -1)
    X_test_flat  = X_test.numpy().reshape(X_test.shape[0], -1)

    # Prediction function for SHAP
    def predict_fn(x):
        x_torch = torch.FloatTensor(x).reshape(-1, params['seq_length'], len(params['features']))
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.eval()
        with torch.no_grad():
            preds = model(x_torch.to(device)).cpu().numpy()
        return preds

    # Select background and samples to explain
    background = X_train_flat[:500]
    X_explain  = X_test_flat

    # SHAP KernelExplainer
    explainer = shap.KernelExplainer(predict_fn, background)
    shap_values = explainer.shap_values(X_explain, nsamples=100)

    # Reshape SHAP values to (samples, seq_length, features)
    shap_values_reshaped = np.array(shap_values).reshape(X_explain.shape[0], params['seq_length'], len(params['features']))

    # Aggregate importance across samples and timesteps
    feature_importance = np.mean(np.abs(shap_values_reshaped), axis=(0, 1))

    # Plot
    feature_names = params['features']
    sorted_idx = np.argsort(feature_importance)[::-1]
    plt.figure(figsize=(10, 6))
    plt.barh(np.array(feature_names)[sorted_idx], feature_importance[sorted_idx])
    plt.xlabel("Mean |SHAP value|")
    plt.title("Feature Importance (Transformer)")
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()